# <h1 align="center"><font color="red">How to Build a Simple Reasoning and Acting Agent from Scratch</font></h1>

<font color="yellow">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este Notebook está baseado no Tutorial de [kirouane Ayoub]().

Links de estudo:

* [Part 1](https://blog.gopenai.com/introduction-to-llm-agents-how-to-build-a-simple-reasoning-and-acting-agent-from-scratch-part-1-843e14686be7)

* [Part 2](https://blog.gopenai.com/building-llm-agents-from-scratch-part-2-a-conversational-search-agent-with-ollama-a4544b2291cf)

# <font color="gree">Contextualizando</font>

Esta Notebook inicia uma série sobre a construção de `agentes de IA´, começando com uma implementação básica usando `Ollama´ para executar `LLMs´ localmente. O objetivo é criar um sistema que compreenda consultas do usuário e utilize pesquisa na web para fornecer respostas informativas, demonstrando os conceitos fundamentais do design de agentes de IA.

Esta implementação serve como um ponto de partida para entender o desenvolvimento de `agentes de IA`, mas é uma versão simplificada. Em cenários reais, os agentes teriam comportamentos mais complexos, exigindo técnicas de análise sintática robustas para lidar com saídas inesperadas. Além disso, seriam necessárias estratégias abrangentes de tratamento de erros para garantir uma operação suave e confiável do sistema.

# <font color="gree">Funcionamento do Agent</font>

![](https://miro.medium.com/v2/resize:fit:828/format:webp/1*-c8XfC8zGlrmF8DMwWkJaA.png)